# PRMT-2355 Vision 'Core extract not sent' transfers July & Half August 2021

## Context
Vision have got back to us regarding ‘pending’ transfers, and would like a list of transfers for them to look into to help diagnose the issue. ‘Pending’ transfers in this context are transfers that have gotten stuck in transit, have no error code(s) but have failed to be received by the receiving practice (i.e. this would include: the transfer outcome failure type as ‘core extract not sent’). 

## Scope
Generate list of transfers for July (14 day cut off) and beginning of August where Vision is the sender and the failure type is ‘core extract not sent’ with no error codes. 

### List should have:
* Sending NACs
* Recieving NACs
* Recieving NACs supplier 
* Time stamp
* Conversation ID


In [1]:
import pandas as pd
from datetime import datetime

In [2]:
transfer_files = [
    "s3://prm-gp2gp-transfer-data-preprod/v4/2021/7/transfers.parquet",
    "s3://prm-gp2gp-notebook-data-prod/PRMT-2355-half-august-data-with-14-day-cutoff/transfers/v4/2021/8/transfers.parquet"
]
transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_files
))

transfers = transfers_raw.copy()

In [3]:
# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

# Generate ASID lookup that contains all the most recent entry for all ASIDs encountered
asid_lookup_file_location = "s3://prm-gp2gp-asid-lookup-preprod/"
asid_lookup_files = [
    "2021/7/asidLookup.csv.gz",
    "2021/8/asidLookup.csv.gz",
    "2021/9/asidLookup.csv.gz"    
]
asid_lookup_input_files = [asid_lookup_file_location + f for f in asid_lookup_files]
asid_lookup = pd.concat((
    pd.read_csv(f)
    for f in asid_lookup_input_files
)).drop_duplicates().groupby("ASID").last().reset_index()
lookup = asid_lookup[["ASID", "NACS","OrgName"]]

transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'ASID': 'requesting_supplier_asid', 'NACS': 'requesting_ods_code','OrgName':'requesting_practice_name'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'ASID': 'sending_supplier_asid', 'NACS': 'sending_ods_code','OrgName':'sending_practice_name'}, axis=1)

# Making the status to be more human readable here
transfers["status"] = transfers["status"].str.replace("_", " ").str.title()

In [4]:
# filter data to just include the first 2 weeks (15 days) of august
date_filter_bool = transfers["date_requested"] < datetime(2021, 8, 16)
transfers = transfers[date_filter_bool]


In [5]:
# filter for Vision as the sender wi
vision_sender_bool = transfers["sending_supplier"]=="Vision"
core_extract_not_sent_bool = transfers["failure_reason"]=="Core Extract not Sent"
vision_sender_where_core_extract_not_sent = transfers[vision_sender_bool & core_extract_not_sent_bool].copy()

In [6]:
sorted_vision_sender_where_core_extract_not_sent = vision_sender_where_core_extract_not_sent.sort_values(by="date_requested", ascending=False).reset_index()
columns_to_keep = ['conversation_id', 'date_requested', "requesting_ods_code", "requesting_practice_name", "sending_ods_code", "sending_practice_name", 'requesting_supplier', 'sending_supplier']
sorted_vision_sender_where_core_extract_not_sent = sorted_vision_sender_where_core_extract_not_sent[columns_to_keep]
sorted_vision_sender_where_core_extract_not_sent

,conversation_id,date_requested,requesting_ods_code,requesting_practice_name,sending_ods_code,sending_practice_name,requesting_supplier,sending_supplier
0,8C43AC4A-FEDF-43CA-8F3D-C6247DF60812,2021-08-13 17:14:20.994,N83614,ECCLESTON MEDICAL CENTRE,P83007,RADCLIFFE MEDICAL PRACTICE,EMIS,Vision
1,EC637C00-FC53-11EB-9013-D902B092ADE1,2021-08-13 16:31:34.546,B82034,QUAKERS LANE SURGERY,P83608,THE ELMS MEDICAL CENTRE,SystmOne,Vision
2,C3BFA22D-0C93-455D-98DE-A11A8027B959,2021-08-13 14:41:25.263,M83046,BIDDULPH VALLEY SURGERY,P83020,MINDEN FAMILY PRACTICES - DR SAXENA,EMIS,Vision
3,093A5473-EC45-46E4-A2AA-7D346C308C06,2021-08-13 14:17:26.400,P86009,CASTLETON HEALTH CENTRE,P83012,TOWER FAMILY HEALTHCARE,EMIS,Vision
4,1BD4DDEE-BEA5-47F9-B163-1BE96B35B3B8,2021-08-13 14:01:27.852,P83012,TOWER FAMILY HEALTHCARE,P83015,RIBBLESDALE MEDICAL PRACTICE,Vision,Vision
...,...,...,...,...,...,...,...,...
928,4C3B07CA-1247-4FAF-93CA-16932FDA07B9,2021-07-01 08:22:07.762,P88007,CHEADLE HULME MEDICAL GROUP,P87620,1/MONTON MEDICAL PRACTICE,EMIS,Vision
929,820F3C80-DA44-11EB-A2A7-99E05E38914D,2021-07-01 08:15:34.593,E85125,STERNDALE SURGERY,G83651,WOODLANDS SURGERY,SystmOne,Vision
930,86D5F72A-EBB0-46FF-9251-0E7EBACB6E13,2021-07-01 07:36:39.232,H81020,FORT HOUSE SURGERY,H84002,PARK ROAD SURGERY,EMIS,Vision
931,EABA5945-16DC-4097-96AB-467E4C842635,2021-07-01 07:34:09.996,H81020,FORT HOUSE SURGERY,H84002,PARK ROAD SURGERY,EMIS,Vision


In [7]:
with pd.ExcelWriter("Vision 'Core extract not sent' transfers July & August 2021 PRMT-2355.xlsx") as writer:
    sorted_vision_sender_where_core_extract_not_sent.to_excel(writer, sheet_name="All",index=False)